In [1]:
import pandas as pd
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

In [2]:
def clean_dataframe_for_arrow(df):
      """
      Clean DataFrame to make it compatible with PyArrow conversion
      """
      print("Cleaning DataFrame for PyArrow compatibility...")

      # Keep only the essential columns for training
      essential_columns = [
          'report_id', 'mapped_clinic_id', 'grouped_modality', 'clinic_modality',
          'training_input', 'training_output', 'clean_findings', 'clean_impression',
          'clean_history', 'clean_technique'
      ]

      # Filter to only columns that exist in the dataframe
      available_columns = [col for col in essential_columns if col in df.columns]
      df_clean = df[available_columns].copy()

      # Convert all columns to string type to avoid Arrow conversion issues
      for col in df_clean.columns:
          if col not in ['report_id']:  # Keep report_id as is if it's numeric
              df_clean[col] = df_clean[col].astype(str)
              # Replace 'nan' strings with None
              df_clean[col] = df_clean[col].replace(['nan', 'None', ''], None)

      # Fill any remaining NaN values
      df_clean = df_clean.fillna('')

      print(f"Cleaned DataFrame shape: {df_clean.shape}")
      print(f"Columns retained: {list(df_clean.columns)}")

      return df_clean

In [3]:
def load_data_to_huggingface():
      """
      Convert enhanced processed radiology data to HuggingFace Dataset format
      """

      # Define paths
      data_dir = "./data"

      # Load the enhanced datasets
      print("Loading enhanced processed data...")
      train_df = pd.read_csv(f"{data_dir}/train_data.csv", low_memory=False)
      val_df = pd.read_csv(f"{data_dir}/val_data.csv", low_memory=False)
      test_df = pd.read_csv(f"{data_dir}/test_data.csv", low_memory=False)

      # Load reference banks for style learning
      with open(f"{data_dir}/reference_banks.json", 'r') as f:
          reference_banks = json.load(f)


      # Radiology-specific system message
      system_message = """You are an expert radiologist assistant specializing in generating accurate and concise medical impressions from radiology
               findings.
            
              Your task is to:
              1. **Analyze the findings**: Carefully review all clinical findings, history, and technique information
              2. **Generate focused impressions**: Create clear, prioritized conclusions that directly address the clinical question
              3. **Maintain clinical accuracy**: Ensure all significant findings are appropriately characterized
              4. **Use appropriate medical terminology**: Follow standard radiological reporting conventions
              5. **Adapt communication style**: Match the institutional reporting style and level of detail expected
            
              Generate only the IMPRESSION section based on the provided clinical information."""

      def process_dataframe_to_chat_format(df):
          """Convert DataFrame directly to chat format"""

          processed_data = []

          for idx, row in df.iterrows():
              try:
                  # Extract fields with fallbacks
                  clinic = str(row.get('mapped_clinic_id', 'unknown'))
                  modality = str(row.get('grouped_modality', 'unknown'))
                  findings_input = str(row.get('training_input', ''))
                  impression_output = str(row.get('training_output', ''))
                  clinic_modality = str(row.get('clinic_modality', f"{clinic}_{modality}"))

                  # Skip if essential fields are missing
                  if (not findings_input or findings_input == 'nan' or
                      not impression_output or impression_output == 'nan'):
                      continue

                  # Create user message with clinical context
                  user_content = f"Please generate an appropriate radiology impression for this {modality} study from {clinic}:\n\n{findings_input}"

                  # Create chat messages
                  messages = [
                      {"role": "system", "content": system_message},
                      {"role": "user", "content": user_content},
                      {"role": "assistant", "content": impression_output}
                  ]

                  processed_data.append({
                      "messages": messages,
                      "clinic_id": clinic,
                      "modality": modality,
                      "findings": findings_input,
                      "impression": impression_output,
                      "clinic_modality": clinic_modality
                  })

              except Exception as e:
                  print(f"Warning: Error processing row {idx}: {e}")
                  continue

          return processed_data

      # Process each DataFrame
      print("Processing DataFrames to chat format...")
      train_data = process_dataframe_to_chat_format(train_df)
      val_data = process_dataframe_to_chat_format(val_df)
      test_data = process_dataframe_to_chat_format(test_df)


      if train_data:
          train_dataset = Dataset.from_list(train_data)
      else:
          print("Warning: No valid training data found!")
          return None, None

      if val_data:
          val_dataset = Dataset.from_list(val_data)
      else:
          print("Warning: No valid validation data found!")
          val_dataset = Dataset.from_list([train_data[0]])  # Use first training sample as fallback

      if test_data:
          test_dataset = Dataset.from_list(test_data)
      else:
          print("Warning: No valid test data found!")
          test_dataset = Dataset.from_list([train_data[0]])  # Use first training sample as fallback

      # Create DatasetDict
      dataset_dict = DatasetDict({
          "train": train_dataset,
          "validation": val_dataset,
          "test": test_dataset
      })

      print("Dataset conversion complete!")
      print(f"Final dataset structure:")
      print(dataset_dict)

      return dataset_dict, reference_banks

In [4]:
def apply_chat_template_to_dataset(dataset_dict, tokenizer, max_length=2048):
      """
      Apply tokenizer chat template to convert messages to model input format
      """

      def process_single_example(example):
          """Process a single example"""
          try:
              messages = example["messages"]

              # Apply chat template
              text = tokenizer.apply_chat_template(
                  messages,
                  tokenize=False,
                  add_generation_prompt=False
              )

              return {
                  "findings": example["findings"],
                  "impression": example["impression"],
                  "text": text,
                  "clinic_id": example["clinic_id"],
                  "modality": example["modality"],
                  "clinic_modality": example["clinic_modality"]
              }

          except Exception as e:
              print(f"Error processing example: {e}")
              return None

      print("Applying chat template and tokenization...")

      # Process each split
      processed_datasets = {}

      for split_name, dataset in dataset_dict.items():
          print(f"Processing {split_name} split...")

          processed_examples = []
          for example in dataset:
              processed = process_single_example(example)
              if processed is not None:
                  processed_examples.append(processed)

          if processed_examples:
              processed_datasets[split_name] = Dataset.from_list(processed_examples)
          else:
              print(f"Warning: No valid examples in {split_name} split!")

      tokenized_dataset_dict = DatasetDict(processed_datasets)

      print("Tokenization complete!")
      print(f"Tokenized dataset structure:")
      print(tokenized_dataset_dict)

      return tokenized_dataset_dict

def prepare_style_reference_data(reference_banks, tokenizer):
  """
  Prepare reference data for style learning during training
  """

  print("Preparing style reference data...")

  style_references = {}

  for clinic_id, impressions in reference_banks.items():
      # Filter out empty impressions
      valid_impressions = [str(imp).strip() for imp in impressions
                         if imp and str(imp).strip() and str(imp) != 'nan']

      if not valid_impressions:
          print(f"Warning: No valid impressions for clinic {clinic_id}")
          continue

      style_references[clinic_id] = {
          "impressions": valid_impressions[:50]  # Limit to 50 for memory efficiency
      }

  print(f"Style references prepared for {len(style_references)} clinics")
  return style_references

In [5]:
# Load model name (same as in original notebook)
model_name = "microsoft/MediPhi-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [6]:
def load_radiology_datasets():
      """
      Main function to load and prepare all radiology datasets for training
      """

      # Load model name
      model_name = "microsoft/MediPhi-Instruct"
      tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

      # Ensure tokenizer has pad token
      if tokenizer.pad_token is None:
          tokenizer.pad_token = tokenizer.eos_token

      # Load and convert data
      dataset_dict, reference_banks = load_data_to_huggingface()

      if dataset_dict is None:
          print("Failed to load datasets!")
          return None, None, None

      # Apply chat template and tokenization
      tokenized_datasets = apply_chat_template_to_dataset(dataset_dict, tokenizer)

      # Prepare style references
      style_references = prepare_style_reference_data(reference_banks, tokenizer)

      # Save processed datasets for later use
      print("Saving processed datasets...")
      tokenized_datasets.save_to_disk("./data/processed/radiology_datasets")

      with open("./data/processed/style_references.json", 'w') as f:
          # Save just the text data
          text_only_refs = {k: v["impressions"] for k, v in style_references.items()}
          json.dump(text_only_refs, f, indent=2)

      print("All datasets saved successfully!")

      return tokenized_datasets, style_references, tokenizer

In [7]:
dataset_dict, reference_banks, tokenizer = load_radiology_datasets()

Loading enhanced processed data...
Processing DataFrames to chat format...
Dataset conversion complete!
Final dataset structure:
DatasetDict({
    train: Dataset({
        features: ['messages', 'clinic_id', 'modality', 'findings', 'impression', 'clinic_modality'],
        num_rows: 8865
    })
    validation: Dataset({
        features: ['messages', 'clinic_id', 'modality', 'findings', 'impression', 'clinic_modality'],
        num_rows: 1901
    })
    test: Dataset({
        features: ['messages', 'clinic_id', 'modality', 'findings', 'impression', 'clinic_modality'],
        num_rows: 1915
    })
})
Applying chat template and tokenization...
Processing train split...
Processing validation split...
Processing test split...
Tokenization complete!
Tokenized dataset structure:
DatasetDict({
    train: Dataset({
        features: ['findings', 'impression', 'text', 'clinic_id', 'modality', 'clinic_modality'],
        num_rows: 8865
    })
    validation: Dataset({
        features: ['findi

Saving the dataset (0/1 shards):   0%|          | 0/8865 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1901 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1915 [00:00<?, ? examples/s]

All datasets saved successfully!


In [8]:
# reference_banks
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['findings', 'impression', 'text', 'clinic_id', 'modality', 'clinic_modality'],
        num_rows: 8865
    })
    validation: Dataset({
        features: ['findings', 'impression', 'text', 'clinic_id', 'modality', 'clinic_modality'],
        num_rows: 1901
    })
    test: Dataset({
        features: ['findings', 'impression', 'text', 'clinic_id', 'modality', 'clinic_modality'],
        num_rows: 1915
    })
})